# Module 9: System Integration

This is the most important module yet. You will combine the two separate systems you have mastered—CAN bus telemetry and AI vision—into a single, intelligent application. This is **Sensor Fusion**.

## Section 1: Your Mission

Build a **Smart Safety Monitor** that fuses camera detections and VESC telemetry.
When risk conditions are met, trigger bounded brake intervention and an alert.


### What Success Looks Like:
- Person detected with low speed: caution only.
- Person detected with medium speed: warning + single beep.
- Person detected with high speed: warning + bounded brake sequence.
- No person detected: no intervention.


## Section 2: Combining Data

To achieve our goal, we must initialize and read from both the `VESCStudentAPI` and the `AIStudentAPI` in the same script.

In [1]:
# 2.1 - Full System Initialization
import sys, os, time
from gpiozero import Buzzer
from IPython.display import clear_output
sys.path.append(os.path.abspath('../'))
from student_api import VESCStudentAPI, AIStudentAPI

# Initialize Buzzer
buzzer = Buzzer(17)

# Initialize AI System
ai_api = AIStudentAPI()
ai_api.start_camera()
print("✅ AI Camera is live.")

# Initialize VESC System
vesc_api = VESCStudentAPI()
vesc = None
if vesc_api.start():
    controllers = []
    deadline = time.time() + 10.0
    while time.time() < deadline and not controllers:
        controllers = vesc_api.get_connected_controllers()
        if not controllers:
            time.sleep(0.5)

    if controllers:
        vesc = vesc_api.get_controller(controllers[0])
        print(f'✅ Connected to VESC: {vesc.controller_id}')
    else:
        print('❌ VESC: No controllers found within 10 seconds')
else:
    print('❌ VESC: Failed to start API')


[0:03:56.622839092] [2044]  INFO Camera camera_manager.cpp:326 libcamera v0.5.0+59-d83ff0a4
[0:03:57.163429741] [2081]  WARN RPiSdn sdn.cpp:40 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[0:03:57.196569089] [2081]  INFO RPI vc4.cpp:447 Registered camera /base/soc/i2c0mux/i2c@1/imx500@1a to Unicam device /dev/media3 and ISP device /dev/media0
[0:03:57.198688988] [2081]  INFO RPI pipeline_base.cpp:1121 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'

------------------------------------------------------------------------------------------------------------------
NOTE: Loading network firmware onto the IMX500 can take several minutes, please do not close down the application.
------------------------------------------------------------------------------------------------------------------

Network Firmware Upload: 0.00bytes [00:00, ?bytes/s][0:03:59.924350945] [2044]  INFO Camera camera.cpp:1205 configuring streams: (0) 640x480-X

✅ AI Camera is live.
Starting VESC CAN System...
Connected to CAN bus: can0
Main processing loop started
VESC CAN System started successfully
❌ VESC: No controllers found within 10 seconds


### EXERCISE: Combined Data Loop
In the cell below, create a `while` loop that, in each iteration, reads the **RPM** from the VESC and gets the **detections** from the AI camera. Print both values on the same line. Run it for 10 seconds. (Don't worry about displaying the video frame).

In [ ]:
start_time = time.time()
try:
    while time.time() - start_time < 10:
        # TODO: read RPM from vesc (if available)
        # TODO: get detections from ai_api
        # TODO: print both on one line

        time.sleep(0.2)
        clear_output(wait=True)

except KeyboardInterrupt:
    print("Loop stopped.")



## Section 3: Conditional Safety Logic

Now we add the core of our smart system: the multi-condition `if` statement.

In [ ]:
# 3.1 - The Core Logic
SPEED_THRESHOLD = 500
CONFIDENCE_THRESHOLD = 0.75
TARGET_LABEL = 'person'

last_alert_time = 0
COOLDOWN = 5

try:
    print("Smart Safety Monitor ACTIVE. Press Interrupt (■) to stop.")
    while True:
        rpm = vesc.get_rpm() if vesc else 0
        _, detections = ai_api.get_frame_and_detections()

        is_person_detected = any(
            det['label'] == TARGET_LABEL and det['confidence'] > CONFIDENCE_THRESHOLD
            for det in detections
        )

        if is_person_detected and abs(rpm) > SPEED_THRESHOLD:
            if time.time() - last_alert_time > COOLDOWN:
                print(f"ALERT! Person detected at {rpm:.0f} RPM")
                buzzer.beep(0.1, 0.1, 2)
                if vesc:
                    vesc.set_brake_current(3.0, 4.0)
                last_alert_time = time.time()

        time.sleep(0.1)

except KeyboardInterrupt:
    print("\nMonitor stopped.")


### EXERCISE: Add a Third Condition
Modify the `if` statement to add a third condition: only alert if the battery voltage is **above 20V**. This prevents alerts when the system is about to shut down.

In [ ]:
# Your code here

### EXERCISE: Create Three Safety Levels
Instead of one alert, create three levels based on speed:
- **Level 1 (Low Risk):** If RPM is between 500-1500, print `Caution`.
- **Level 2 (Medium Risk):** If RPM is between 1501-3000, print `Warning` and beep once.
- **Level 3 (High Risk):** If RPM is > 3000, print `DANGER`, beep three times, and call `set_brake_current(4.0, 3.0)`.



In [ ]:
# Your code with three safety levels here

## Section 4: Knowledge Check

### EXERCISE: Short Answer
In your own words, what is **Sensor Fusion** and why is it important for building smart systems?

// Your answer here

### EXERCISE: System Improvement
Besides adding more sensors, what is one way you could improve the logic of our Smart Safety Monitor? (e.g., think about the direction of movement, the size of the object, etc.)

// Your answer here

---
## Congratulations! You've completed Module 9.

In [ ]:
# Cleanly stop all APIs and release hardware
if 'ai_api' in locals():
    ai_api.stop_camera()
    print("Camera stopped.")
if 'vesc_api' in locals() and vesc_api.is_running():
    vesc_api.stop()
    print("VESC API connection closed.")
if 'buzzer' in locals():
    buzzer.close()
    print("Buzzer released.")